<a href="https://colab.research.google.com/github/Shahahaha11/LION_DEN/blob/Linh's-Branch/Copy_of_DATA_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TESTING GEOGRAPHICALLY WEIGHTED REGRESSION . Fotheringham, Brunsdon & Charlton (2003)

In [2]:
import pandas as pd
import numpy as np
file_path= "/content/drive/MyDrive/lions_den_data/austinHousingData.csv"
data = pd.read_csv(file_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#USELESS FEATURES ## BUT WE DROP "city" after using it. ## no dropping yet
useless_features= ['zipcode','zpid', 'city', 'streetAddress', 'zipcode', 'description','homeImage','homeType', 'latestPriceSource']

In [4]:
###################### NEW VARIABLES #################

In [5]:
data['zipcode'].nunique()

48

In [7]:
zp = pd.read_csv("/content/drive/MyDrive/lions_den_data/uszips.csv")
zp.tail()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
33778,99923,55.98043,-130.03803,Hyder,AK,Alaska,True,NaN,25.0,0.6,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33779,99925,55.55398,-132.96276,Klawock,AK,Alaska,True,NaN,854.0,6.1,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33780,99926,55.12617,-131.48928,Metlakatla,AK,Alaska,True,NaN,1385.0,4.0,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Metlakatla
33781,99927,56.33305,-133.60044,Point Baker,AK,Alaska,True,NaN,18.0,1.5,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33782,99929,56.36089,-132.00635,Wrangell,AK,Alaska,True,NaN,2105.0,0.3,2275,Wrangell,"{""02275"": 100}",Wrangell,02275,False,False,America/Sitka


In [8]:
data['latest_saledate'].dtypes

dtype('O')

In [9]:
data['latest_saledate']

,latest_saledate
0,02/09/2019
1,13/10/2020
2,31/07/2019
3,08/08/2018
4,31/10/2018
...,...
15166,10/12/2020
15167,16/12/2020
15168,27/01/2021
15169,07/08/2019


In [10]:
data['latest_saledate'] = data['latest_saledate'].str.strip()
data['latest_saledate'] = pd.to_datetime(data['latest_saledate'], format='%d/%m/%Y', errors='coerce')

print(data['latest_saledate'].isna().sum())

0


In [11]:
# Data inspection
print(f"integer:{data.select_dtypes(include='integer').columns}")
print(f"object:{data.select_dtypes(include='object').columns}")
print(f"bools:{data.select_dtypes(include='bool').columns}")

integer:Index(['zpid', 'zipcode', 'garageSpaces', 'parkingSpaces', 'yearBuilt',
       'latestPrice', 'numPriceChanges', 'latest_salemonth', 'latest_saleyear',
       'numOfPhotos', 'numOfAccessibilityFeatures', 'numOfAppliances',
       'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBedrooms', 'numOfStories'],
      dtype='object')
object:Index(['city', 'streetAddress', 'description', 'homeType', 'latestPriceSource',
       'homeImage'],
      dtype='object')
bools:Index(['hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView'],
      dtype='object')


In [12]:
 # We have 958 days of analysis
 data['latest_saledate'].value_counts().sort_index()

,count
latest_saledate,
2018-01-22,4
2018-01-23,10
2018-01-24,8
2018-01-25,11
2018-01-26,13
...,...
2021-01-26,2
2021-01-27,2
2021-01-28,1


In [13]:
#################### 1 CITY ONLY ###############################
df= data[data['city'] == 'austin'].sort_values(by='latest_saledate')
df.select_dtypes(include='integer').columns

Index(['zpid', 'zipcode', 'garageSpaces', 'parkingSpaces', 'yearBuilt',
       'latestPrice', 'numPriceChanges', 'latest_salemonth', 'latest_saleyear',
       'numOfPhotos', 'numOfAccessibilityFeatures', 'numOfAppliances',
       'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBedrooms', 'numOfStories'],
      dtype='object')

In [14]:
data['latest_saledate'].dropna()

,latest_saledate
0,2019-09-02
1,2020-10-13
2,2019-07-31
3,2018-08-08
4,2018-10-31
...,...
15166,2020-12-10
15167,2020-12-16
15168,2021-01-27
15169,2019-08-07


In [15]:
object_features = ['city', 'streetAddress', 'description', 'homeType', 'latestPriceSource','homeImage']

In [16]:
df.select_dtypes(include='object').columns

Index(['city', 'streetAddress', 'description', 'homeType', 'latestPriceSource',
       'homeImage'],
      dtype='object')

In [17]:
################# FINAL DATAFRAME IN USE ################
df_1 = df.drop(columns=useless_features, errors='ignore')
df_1.shape

(15020, 39)

In [18]:
# FINAL CHECK BEFORE MERGING
df_1.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [20]:
# Get Mortgage rate
import pandas as pd
MTG = pd.read_csv('/content/drive/MyDrive/lions_den_data/MORTGAGE30US.csv')

print(MTG.shape)

MPPSQ = pd.read_csv('/content/drive/MyDrive/lions_den_data/MEDLISPRIPERSQUFEE12420.csv')
print(MPPSQ.dtypes)

(160, 2)
observation_date           object
MEDLISPRIPERSQUFEE12420     int64
dtype: object


In [30]:
#Get air_quality
import pandas as pd
air_2018 = pd.read_csv('/content/ad_viz_plotval_data_2018.csv')
air_2019 = pd.read_csv('/content/ad_viz_plotval_data_2019.csv')
air_2020 = pd.read_csv('/content/ad_viz_plotval_data_2020.csv')
air_2021 = pd.read_csv('/content/ad_viz_plotval_data_2021.csv')
AIR = pd.concat([air_2018, air_2019, air_2020, air_2021], axis=0, ignore_index=True)
print(AIR.shape)

(467, 21)


In [31]:
# Delete unnecessary columns
AIR = AIR.loc[:,['Date','Daily Mean PM10 Concentration', 'Units', 'Daily AQI Value', 'Local Site Name']]
print(AIR.head())

         Date  Daily Mean PM10 Concentration     Units  Daily AQI Value  \
0  01/02/2018                             10  ug/m3 SC                9   
1  01/08/2018                             24  ug/m3 SC               22   
2  01/14/2018                             11  ug/m3 SC               10   
3  01/20/2018                              7  ug/m3 SC                6   
4  01/26/2018                             20  ug/m3 SC               19   

          Local Site Name  
0  Austin Audubon Society  
1  Austin Audubon Society  
2  Austin Audubon Society  
3  Austin Audubon Society  
4  Austin Audubon Society  


In [32]:
AIR['Date'] = pd.to_datetime(AIR['Date'], format='%m/%d/%Y', errors='coerce')  # Fix air data format
print(AIR['Date'])

0     2018-01-02
1     2018-01-08
2     2018-01-14
3     2018-01-20
4     2018-01-26
         ...    
462   2021-12-06
463   2021-12-12
464   2021-12-18
465   2021-12-24
466   2021-12-30
Name: Date, Length: 467, dtype: datetime64[ns]


In [33]:
import pandas as pd

# Convert all date columns to datetime format
df_2['latest_saledate'] = pd.to_datetime(df_2['latest_saledate'], errors='coerce', format='%m-%d-%Y')
AIR['Date'] = pd.to_datetime(AIR['Date'], errors='coerce', format='%m-%d-%Y')

# Check for any NaT (missing values) in the date columns
print(df_2['latest_saledate'].isna().sum())
print(MTG['observation_date'].isna().sum())
print(MPPSQ['observation_date'].isna().sum())
print(AIR['Date'].isna().sum())

0
0
0
0


In [ ]:
############################################################################### Tranforming Mortgage Data #############################################################################################

In [21]:
################## TREATING DATES ON EXTERNAL MACRO VARIABLES ##################
def parse_obs_date(date_str):
    d = str(date_str).strip().split()[0]
    if "/" in d:
        return pd.to_datetime(d, format='%d/%m/%Y', errors='raise')
    else:
        return pd.to_datetime(d, errors='raise')

MTG['observation_date'] = MTG['observation_date'].apply(parse_obs_date)
MPPSQ['observation_date'] = MPPSQ['observation_date'].apply(parse_obs_date)
MTG['MORTGAGE30US'].dtypes

dtype('float64')

In [22]:
print(MTG.isna().sum())
print(MPPSQ.isna().sum())

observation_date    0
MORTGAGE30US        0
dtype: int64
observation_date           0
MEDLISPRIPERSQUFEE12420    0
dtype: int64


In [23]:
df_2 = df_1.copy()

In [24]:
##################################.MERGE. ####################################

import pandas as pd

df_2['latest_saledate'] = pd.to_datetime(df_2['latest_saledate'], errors='coerce')
df_2 = df_2.sort_values('latest_saledate')

# Read and prepare the MTG rates
MTG['observation_date'] = pd.to_datetime(MTG['observation_date'], format='%d/%m/%Y', errors='coerce')
MTG = MTG.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MTG rate available
df_2 = pd.merge_asof(df_2, MTG, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'MORTGAGE30US'],
      dtype='object')

In [25]:
# read variable, sort variable on date
MPPSQ['observation_date'] = pd.to_datetime(MPPSQ['observation_date'], format='%d/%m/%Y', errors='coerce')
MPPSQ = MPPSQ.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MPPSQ available
df_2 = pd.merge_asof(df_2, MPPSQ, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if you no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'MORTGAGE30US',
       'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

In [34]:
# Read and prepare the AIR for df_2
AIR['Date'] = pd.to_datetime(AIR['Date'], format='%d/%m/%Y', errors='coerce')
AIR = AIR.sort_values('Date')

# Merge using asof so that each sale date gets the most recent MPPSQ available
df_2 = pd.merge_asof(df_2, AIR, left_on='latest_saledate', right_on='Date', direction='backward')

# Optional: Remove the duplicate 'date' column if you no longer need it
df_2.drop(columns=['Date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'MORTGAGE30US',
       'MEDLISPRIPERSQUFEE12420', 'Daily Mean PM10 Concentration', 'Units',
       'Daily AQI Value', 'Local Site Name'],
      dtype='object')

In [35]:
df_2[['latest_saledate','MORTGAGE30US','MEDLISPRIPERSQUFEE12420', 'Daily AQI Value']]

,latest_saledate,MORTGAGE30US,MEDLISPRIPERSQUFEE12420,Daily AQI Value
0,2018-01-22,4.04,163,14
1,2018-01-22,4.04,163,14
2,2018-01-22,4.04,163,14
3,2018-01-22,4.04,163,14
4,2018-01-23,4.04,163,14
...,...,...,...,...
15015,2021-01-27,2.77,226,14
15016,2021-01-28,2.73,226,14
15017,2021-01-29,2.73,226,14
15018,2021-01-29,2.73,226,14


In [36]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
df_2.to_pickle("/content/drive/MyDrive/lions_den_data/df_2.pkl")

In [ ]:
## run -> on next script -> df_2 = pd.read_pickle('/content/drive/MyDrive/lions_den_data/df_2.pkl')
